In [ ]:
from stockfish import Stockfish
import chess
import csv
import os
from concurrent.futures import ThreadPoolExecutor

# Define Stockfish parameters
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"

def create_stockfish_instance():
    return Stockfish(
        path=STOCKFISH_PATH,
        depth=18,
        parameters={
            "Threads": 2,
            "Minimum Thinking Time": 5,  # in miliseconds
        }
    )

# 1 game
def play_game(stockfish_instance):
    board = chess.Board()
    moves = []
    
    while not board.is_game_over():
        stockfish_instance.set_fen_position(board.fen())
        best_move = stockfish_instance.get_best_move()
        moves.append(board.san(chess.Move.from_uci(best_move)))  #  move in SAN format
        board.push(chess.Move.from_uci(best_move))
    
    #  game result
    outcome = board.outcome()
    if outcome is not None:
        if outcome.winner is True:
            winner = "White"
        elif outcome.winner is False:
            winner = "Black"
        else:
            winner = "Draw"
    else:
        winner = "Unknown"

    return winner, moves

# Get the starting game number
def get_starting_game_number(csv_file):
    if not os.path.exists(csv_file):
        return 1
    with open(csv_file, "r") as file:
        reader = csv.reader(file)
        rows = list(reader)
        if len(rows) <= 1:
            return 1  # Only header or perhaps an empty file
        last_row = rows[-1]
        return int(last_row[0]) + 1  # Next game number

# save new data to csv
def generate_games_dataset(num_games=1000, num_threads=3):
    csv_file = "ai_chess_games.csv"
    starting_game_number = get_starting_game_number(csv_file)
    
    with open(csv_file, "a", newline="") as csvfile:
        fieldnames = ["game_number", "winner", "moves"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if starting_game_number == 1:
            writer.writeheader()  # write header if file new

        # play games in parallel
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = []
            for game_number in range(starting_game_number, starting_game_number + num_games):
                stockfish_instance = create_stockfish_instance()
                futures.append(executor.submit(play_and_record_game, stockfish_instance, writer, game_number))

            # Print results
            for future in futures:
                future.result()  # (this ensurse each game completes)

def play_and_record_game(stockfish_instance, writer, game_number):
    winner, moves = play_game(stockfish_instance)
    writer.writerow({
        "game_number": game_number,
        "winner": winner,
        "moves": " ".join(moves)  # join moves as a single string
    })
    print(f"Game {game_number} complete. Winner: {winner}")

# run the dataset generation
generate_games_dataset(num_games=50, num_threads=3)


Game 6 complete. Winner: Draw
Game 4 complete. Winner: Draw
Game 5 complete. Winner: Draw
Game 7 complete. Winner: Draw
Game 9 complete. Winner: Draw
Game 8 complete. Winner: Draw
Game 11 complete. Winner: Draw
Game 10 complete. Winner: Draw
Game 12 complete. Winner: Draw
Game 15 complete. Winner: Draw
Game 13 complete. Winner: White
Game 14 complete. Winner: White
Game 16 complete. Winner: Draw
Game 18 complete. Winner: Draw
Game 17 complete. Winner: Draw
Game 19 complete. Winner: Draw
Game 20 complete. Winner: White
Game 22 complete. Winner: Draw
Game 21 complete. Winner: White
Game 24 complete. Winner: Draw
Game 23 complete. Winner: Draw
Game 26 complete. Winner: Draw
Game 27 complete. Winner: Draw
Game 25 complete. Winner: White
Game 29 complete. Winner: Draw
Game 30 complete. Winner: Draw
Game 28 complete. Winner: Draw
Game 31 complete. Winner: Draw
Game 32 complete. Winner: Draw
Game 33 complete. Winner: Draw
Game 34 complete. Winner: White
Game 35 complete. Winner: Draw
Game 36 

In [ ]:
from stockfish import Stockfish
import chess
import csv
import os
from concurrent.futures import ThreadPoolExecutor

# Define Stockfish parameters
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"  # Replace with your Stockfish binary path

def create_stockfish_instance():
    return Stockfish(
        path=STOCKFISH_PATH,
        depth=18,
        parameters={
            "Threads": 2,
            "Minimum Thinking Time": 5, 
        }
    )


def play_game_with_d4(stockfish_instance):
    board = chess.Board()
    moves = ["d4"]  # Start with d4 as the first move
    board.push_san("d4")  # Push d4 to the board
    
    while not board.is_game_over():
        stockfish_instance.set_fen_position(board.fen())
        best_move = stockfish_instance.get_best_move()
        moves.append(board.san(chess.Move.from_uci(best_move)))  # SAN format
        board.push(chess.Move.from_uci(best_move))
    
    #  game result
    outcome = board.outcome()
    if outcome is not None:
        if outcome.winner is True:
            winner = "White"
        elif outcome.winner is False:
            winner = "Black"
        else:
            winner = "Draw"
    else:
        winner = "Unknown"

    return winner, moves


def get_starting_game_number(csv_file):
    if not os.path.exists(csv_file):
        return 1
    with open(csv_file, "r") as file:
        reader = csv.reader(file)
        rows = list(reader)
        if len(rows) <= 1:
            return 1  
        last_row = rows[-1]
        return int(last_row[0]) + 1 

# Generate dataset  &  save to CSV
def generate_games_dataset(num_games=20, num_threads=3):
    csv_file = "ai_chess_games.csv"
    starting_game_number = get_starting_game_number(csv_file)
    
    with open(csv_file, "a", newline="") as csvfile:
        fieldnames = ["game_number", "winner", "moves"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if starting_game_number == 1:
            writer.writeheader()  # header only if the file is new

        # play games in parallel
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = []
            for game_number in range(starting_game_number, starting_game_number + num_games):
                stockfish_instance = create_stockfish_instance()
                futures.append(executor.submit(play_and_record_game_with_d4, stockfish_instance, writer, game_number))

            # Print result
            for future in futures:
                future.result()  

def play_and_record_game_with_d4(stockfish_instance, writer, game_number):
    winner, moves = play_game_with_d4(stockfish_instance)
    writer.writerow({
        "game_number": game_number,
        "winner": winner,
        "moves": " ".join(moves)  
    })
    print(f"Game {game_number} complete. Winner: {winner}")

generate_games_dataset(num_games=20, num_threads=3)


Game 56 complete. Winner: Draw
Game 55 complete. Winner: Draw
Game 54 complete. Winner: Draw
Game 57 complete. Winner: Draw
Game 58 complete. Winner: Draw
Game 59 complete. Winner: Draw
Game 61 complete. Winner: Draw
Game 62 complete. Winner: Draw
Game 63 complete. Winner: Draw
Game 64 complete. Winner: Draw
Game 65 complete. Winner: Draw
Game 66 complete. Winner: White
Game 68 complete. Winner: Draw
Game 67 complete. Winner: Draw
Game 60 complete. Winner: White
Game 69 complete. Winner: Draw
Game 71 complete. Winner: White
Game 70 complete. Winner: Draw
Game 72 complete. Winner: Draw
Game 73 complete. Winner: Draw
